In [1]:
# Check core SDK version number for debugging purposes
import azureml.core
print("SDK Version:", azureml.core.VERSION)

SDK Version: 0.1.0.995245


In [2]:
subscription_id = "786d6510-8f1e-4ae7-b55b-5178716e6ac8"
resource_group = "test-anivan"
workspace_name = "workspace-test1"
workspace_region = 'eastus2' # or eastus2euap

In [3]:
# import the Workspace class and check the azureml SDK version# impor 
from azureml.core.workspace import Workspace, WorkspaceException

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region)
ws.get_details()

{'applicationInsights': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.insights/components/workspacinsightslksuficz',
 'containerRegistry': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.containerregistry/registries/workspacacrezbodeet',
 'description': '',
 'friendlyName': 'workspace-test1',
 'id': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourceGroups/test-anivan/providers/Microsoft.MachineLearningServices/workspaces/workspace-test1',
 'identityPrincipalId': '2059178a-03cf-4cc9-ad26-a5d8ba8ee61b',
 'identityTenantId': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourcegroups/test-anivan/providers/microsoft.keyvault/vaults/workspackeyvaultqkjjihff',
 'location': 'eastus2',
 'name': 'workspace-test1',
 'storageAccount': '/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/

In [42]:
model = ws.register_model(model_path = "model.pkl",
                          model_name = "pdm-multi-class",
                          tags = ["pdm"],
                          description = "Predictive Maintenance multi-class classifier")

Client-Request-ID=af9b4c82-695b-11e8-a4fb-80ee7383b886 Retry policy did not allow for a retry: Server-Timestamp=Wed, 06 Jun 2018 07:31:54 GMT, Server-Request-ID=5f2f6354-101e-00ac-4368-fda0cd000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:5f2f6354-101e-00ac-4368-fda0cd000000Time:2018-06-06T07:31:54.8463753Z</Message></Error>.


Registering model: pdm-multi-class


In [36]:
%%writefile score.py

import json
from sklearn.externals import joblib
import pandas as pd

def init():
    global model
    model = joblib.load("model.pkl")

def run(rawdata):
    try:
        data = json.loads(rawdata)
        df = pd.DataFrame(data=data)
        result = model.predict(df).tolist()
    except Exception as e:
        result = str(e)
    return result

Overwriting score.py


In [10]:
# this forces IPython to load the latest version
# of the score module

import importlib
# https://docs.python.org/3/library/importlib.html#importlib.invalidate_caches
importlib.invalidate_caches()
import score
importlib.reload(score)

<module 'score' from '/home/andrew/work/AI-PredictiveMaintenance/src/AML/Notebooks/score.py'>

In [11]:
score.init()

In [12]:
with open('sample.json', 'r') as sample_file:
  sample_json = sample_file.read()

score.run(sample_json)

['', '', '', '', '']

In [40]:
%%writefile env.yml

name: pdm
channels:
  - defaults
dependencies:
  - pip:
    - pandas==0.22.0

Writing env.yml


In [43]:
image = ws.create_image(name = "pdm-multi-class-image",
                        models = [model],
                        runtime = "python",
                        driver_file = "score.py",
                        conda_file = "env.yml",
                        tags = ["pdm"],
                        description = "Image for Predicitve Maintenance multi-class classifier")

Client-Request-ID=d04932a0-695b-11e8-a4fb-80ee7383b886 Retry policy did not allow for a retry: Server-Timestamp=Wed, 06 Jun 2018 07:32:48 GMT, Server-Request-ID=b7060a9c-101e-00ca-5f68-fd1297000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:b7060a9c-101e-00ca-5f68-fd1297000000Time:2018-06-06T07:32:49.5834223Z</Message></Error>.
Client-Request-ID=d04932a2-695b-11e8-a4fb-80ee7383b886 Retry policy did not allow for a retry: Server-Timestamp=Wed, 06 Jun 2018 07:32:49 GMT, Server-Request-ID=5058cf17-901e-0058-6c68-fd8521000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:5058cf17-901e-0058-6c68-fd852

Creating image


In [44]:
image.wait_for_creation(show_output = True)

Image creation operation finished for image pdm-multi-class-image:1, operation "Succeeded"


In [4]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = ['pdm'], 
                                               description = 'Predicting equipment failure')

In [6]:
image = ws.get_image(name = "pdm-multi-class-image")

In [8]:
service = ws.deploy_webservice(name = 'pdm1',
                               deploy_config = aciconfig,
                               target = None,
                               image = image)
service.wait_for_deployment(show_output = True)

Creating service
Running...............................................
SucceededACI service creation operation finished, operation "Succeeded"


In [13]:
service = ws.get_webservice(name = 'pdm1')
predictions = service.run(input_data = sample_json)
print(predictions)

['', '', '', '', '']


In [15]:
vars(service)

{'_auth': <azureml.core.authentication.InteractiveLoginAuthentication at 0x7f9799d67470>,
 '_mms_endpoint': 'https://eastus2.modelmanagement.azureml.net/api/subscriptions/786d6510-8f1e-4ae7-b55b-5178716e6ac8/resourceGroups/test-anivan/providers/Microsoft.MachineLearningServices/workspaces/workspace-test1/services',
 '_operation_endpoint': None,
 'compute_type': 'ACI',
 'container_resource_requirements': <azureml.core.webservice.aci.ContainerResourceRequirements at 0x7f978f1f86a0>,
 'created_time': datetime.datetime(2018, 6, 6, 7, 44, 51, 850528, tzinfo=tzutc()),
 'description': 'Predicting equipment failure',
 'error': None,
 'image_id': 'pdm-multi-class-image:1',
 'name': 'pdm1',
 'scoring_uri': 'http://40.114.125.83:5001/score',
 'state': None,
 'tags': ['pdm'],
 'updated_time': datetime.datetime(2018, 6, 6, 7, 44, 59, 8069, tzinfo=tzutc()),
 'webservice_id': 'pdm1',
 'workspace': <azureml.core.workspace.Workspace at 0x7f9799ac3278>}